# NCAA Tournament Simulations

This notebook simulates the 2015 NCAA Tournament using our distribution of features.

## Setup

In [47]:
# Libraries.
import numpy as np
import pandas as pd
import pickle

# Custom libraries.
import game_predictions
# game_predictions = reload(game_predictions)

# Library settings.
% matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [42]:
# Paths.
# Pre-tournament statistics path.
stats_path = '../../../data/kenpom_summaries/summary15_pt.csv'
# Bracket paths.
tournament_bracket_path = '../../../data/tournament_brackets/2015_64.csv'
final_four_bracket_path = '../../../data/tournament_brackets/2015_4.csv'

## Data

In [14]:
# Read stats.
pre_tournament_stats = pd.read_csv(stats_path)

In [43]:
# Read brackets.
tournament_bracket = pd.read_csv(bracket_path, header=None).values
final_four_bracket = pd.read_csv(final_four_bracket_path, header=None).values

In [50]:
# Trace.
with open('MH_traces.pkl', 'rb') as fid:
    trace = pickle.load(fid)

In [11]:
# Features.
features = ['diff_Pythag','diff_AdjOE','diff_AdjDE','location_Away','diff_RankAdjOE']

## Simulations

#### Whole-Tournament

In [56]:
# Simulate matchups and outcomes.
tournament_matchups, tournament_outcomes = game_predictions.simulate_tournament(tournament_bracket, pre_tournament_stats, features, coef_trace=trace)

In [57]:
tournament_matchups.head()

,winner,loser,round_of,count
0,Arizona,Baylor,16,9973
1,Arizona,Duke,2,43
2,Arizona,Gonzaga,2,70
3,Arizona,Iowa St.,2,2
4,Arizona,Kentucky,4,435


In [59]:
tournament_matchups[tournament_matchups.round_of == 2].sort(c

,winner,loser,round_of,count
1,Arizona,Duke,2,43
2,Arizona,Gonzaga,2,70
3,Arizona,Iowa St.,2,2
7,Arizona,Utah,2,7
9,Arizona,Villanova,2,55
10,Arizona,Virginia,2,69
31,Duke,Arizona,2,25
36,Duke,Kentucky,2,12
44,Duke,Wisconsin,2,28
48,Gonzaga,Arizona,2,28


In [58]:
tournament_outcomes

,wins_round_1,wins_round_2,wins_round_3,wins_round_4,wins_round_5,wins_round_6
Albany,0,0,0,0,0,0
Arizona,10000,10000,9999,4826,435,246
Arkansas,10000,27,0,0,0,0
Baylor,10000,9974,1,0,0,0
Belmont,0,0,0,0,0,0
Buffalo,63,34,0,0,0,0
Butler,4975,258,72,0,0,0
Cincinnati,5391,0,0,0,0,0
Coastal Carolina,0,0,0,0,0,0
Davidson,4651,1,1,0,0,0


#### Final Four

In [52]:
# Simulate matchups and outcomes.
final_four_matchups, final_four_outcomes = game_predictions.simulate_tournament(final_four_bracket, pre_tournament_stats, features, coef_trace=trace)

In [55]:
final_four_matchups

,winner,loser,round_of,count
0,Duke,Kentucky,2,46
1,Duke,Michigan St.,4,10000
2,Duke,Wisconsin,2,185
3,Kentucky,Duke,2,7118
4,Kentucky,Wisconsin,4,7164
5,Wisconsin,Duke,2,2651
6,Wisconsin,Kentucky,4,2836


In [54]:
final_four_outcomes

,wins_round_1,wins_round_2
Duke,10000,231
Kentucky,7164,7118
Michigan St.,0,0
Wisconsin,2836,2651
